In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
import pandas as pd
import time

In [4]:
url = 'https://nfe.prefeitura.sp.gov.br/'
CNPJ = #INCLUIR O CNPJ - APENAS NUMEROS
password = '' # INCLUIR A SENHA 
faturar = webdriver.Chrome('chromedriver.exe')
wdw = WebDriverWait(faturar, 30)
faturar.get(url)

login_locator = (By.ID, 'ctl00_body_tbCpfCnpj')
CPF_locator = (By.ID, 'ctl00_body_tbCPFCNPJTomador')
description_locator = (By.ID, 'ctl00_body_tbDiscriminacao')
value_locator = (By.ID, 'ctl00_body_tbValor')
voltar_locator = (By.ID, 'ctl00_cphBase_btVoltar')

file_name =  'ExampleSheet.xlsx' #NOME DO ARQUIVO QUE ESTÂO AS NOTAS A SEREM FATURADAS
sheet =  'MesA' #NOME DA ABA A SER FATURADA
df = pd.read_excel(io=file_name, sheet_name=sheet, header=1)

wdw.until(EC.presence_of_element_located(login_locator))
faturar.find_element_by_id('btnAtualizar').click()
time.sleep(3)
login = faturar.find_element_by_id('ctl00_body_tbCpfCnpj')
login.send_keys(CNPJ)
senha = faturar.find_element_by_id('ctl00_body_tbSenha')
senha.send_keys(password)
botao = faturar.find_element_by_id('ctl00_body_btEntrar')
time.sleep(10)
botao.click()
time.sleep(5)
faturar.find_element_by_id('ctl00_wpMenuLateral_mnuRotinasn3').click()
wdw.until(EC.presence_of_element_located(CPF_locator))
for index, row in df.iterrows():    
    valorafaturar = row['VALOR']
    CPF_input = faturar.find_element_by_id('ctl00_body_tbCPFCNPJTomador')
    CPF_input.send_keys(row['CPF / CNPJ'])
    time.sleep(1)
    wdw.until(EC.presence_of_element_located(CPF_locator))
    faturar.find_element_by_id('ctl00_body_btAvancar').click()
    wdw.until(EC.presence_of_element_located(description_locator))
    description_input = faturar.find_element_by_id('ctl00_body_tbDiscriminacao')
    description_input.send_keys(row['DESCRIÇÃO'])
    wdw.until(EC.presence_of_element_located(value_locator))
    value_input = faturar.find_element_by_id('ctl00_body_tbValor')
    if len(str(valorafaturar)) == 3:
        value_input.send_keys(str(valorafaturar) + '00')
    elif len(str(valorafaturar)) == 5:
        value_input.send_keys(str(valorafaturar) + '0')
    else:
        value_input.send_keys(str(valorafaturar))
    time.sleep(2)
    faturar.find_element_by_id('ctl00_body_btEmitir').click()
    time.sleep(3)
    faturar.switch_to.alert.accept()
    wdw.until(EC.presence_of_element_located(voltar_locator))
    faturar.find_element_by_id('ctl00_cphBase_btVoltar').click()